# T1547.006 Kernel Modules and Extensions

-----------------------------------------------------------------------

## Technique Description

Adversaries may modify the kernel to automatically execute programs on system boot. Loadable Kernel Modules (LKMs) are pieces of code that can be loaded and unloaded into the kernel upon demand. They extend the functionality of the kernel without the need to reboot the system. For example, one type of module is the device driver, which allows the kernel to access hardware connected to the system.(Citation: Linux Kernel Programming) 

When used maliciously, LKMs can be a type of kernel-mode [Rootkit](https://attack.mitre.org/techniques/T1014) that run with the highest operating system privilege (Ring 0).(Citation: Linux Kernel Module Programming Guide) Common features of LKM based rootkits include: hiding itself, selective hiding of files, processes and network activity, as well as log tampering, providing authenticated backdoors, and enabling root access to non-privileged users.(Citation: iDefense Rootkit Overview)

Kernel extensions, also called kext, are used in macOS to load functionality onto a system similar to LKMs for Linux. Since the kernel is responsible for enforcing security and the kernel extensions run as apart of the kernel, kexts are not governed by macOS security policies. Kexts are loaded and unloaded through <code>kextload</code> and <code>kextunload</code> commands. Kexts need to be signed with a developer ID that is granted privileges by Apple allowing it to sign Kernel extensions. Developers without these privileges may still sign kexts but they will not load unless SIP is disabled. If SIP is enabled, the kext signature is verified before being added to the AuxKC.(Citation: System and kernel extensions in macOS)

Since macOS Catalina 10.15, kernel extensions have been deprecated in favor of System Extensions. However, kexts are still allowed as "Legacy System Extensions" since there is no System Extension for Kernel Programming Interfaces.(Citation: Apple Kernel Extension Deprecation)

Adversaries can use LKMs and kexts to conduct [Persistence](https://attack.mitre.org/tactics/TA0003) and/or [Privilege Escalation](https://attack.mitre.org/tactics/TA0004) on a system. Examples have been found in the wild, and there are some relevant open source projects as well.(Citation: Volatility Phalanx2)(Citation: CrowdStrike Linux Rootkit)(Citation: GitHub Reptile)(Citation: GitHub Diamorphine)(Citation: RSAC 2015 San Francisco Patrick Wardle)(Citation: Synack Secure Kernel Extension Broken)(Citation: Securelist Ventir)(Citation: Trend Micro Skidmap)

## Technique Detection

Loading, unloading, and manipulating modules on Linux systems can be detected by monitoring for the following commands: <code>modprobe</code>, <code>insmod</code>, <code>lsmod</code>, <code>rmmod</code>, or <code>modinfo</code> (Citation: Linux Loadable Kernel Module Insert and Remove LKMs) LKMs are typically loaded into <code>/lib/modules</code> and have had the extension .ko ("kernel object") since version 2.6 of the Linux kernel. (Citation: Wikipedia Loadable Kernel Module)

Adversaries may run commands on the target system before loading a malicious module in order to ensure that it is properly compiled. (Citation: iDefense Rootkit Overview) Adversaries may also execute commands to identify the exact version of the running Linux kernel and/or download multiple versions of the same .ko (kernel object) files to use the one appropriate for the running system.(Citation: Trend Micro Skidmap) Many LKMs require Linux headers (specific to the target kernel) in order to compile properly. These are typically obtained through the operating systems package manager and installed like a normal package. On Ubuntu and Debian based systems this can be accomplished by running: <code>apt-get install linux-headers-$(uname -r)</code> On RHEL and CentOS based systems this can be accomplished by running: <code>yum install kernel-devel-$(uname -r)</code>

On macOS, monitor for execution of <code>kextload</code> commands and user installed kernel extensions performing abnormal and/or potentially malicious activity (such as creating network connections). Monitor for new rows added in the <code>kext_policy</code> table. KextPolicy stores a list of user approved (non Apple) kernel extensions and a partial history of loaded kernel modules in a SQLite database, <code>/var/db/SystemPolicyConfiguration/KextPolicy</code>.(Citation: User Approved Kernel Extension Pike’s)(Citation: Purves Kextpocalypse 2)(Citation: Apple Developer Configuration Profile)


-----------------------------------------------------------------------

### Tactics:

  * Persistence

  * Privilege-Escalation

### Platforms:

  * macOS

  * Linux

### Adversary Required Permissions:

  * root

### Data Sources:

  * **File:** File Creation

  * **File:** File Modification

  * **Process:** Process Creation

  * **Command:** Command Execution

  * **Kernel:** Kernel Module Load

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1547/006)

  * [Apple Developer Configuration Profile](https://developer.apple.com/business/documentation/Configuration-Profile-Reference.pdf), Apple. (2019, May 3). Configuration Profile Reference. Retrieved September 23, 2021.

  * [Apple Kernel Extension Deprecation](https://developer.apple.com/support/kernel-extensions/), Apple. (n.d.). Deprecated Kernel Extensions and System Extension Alternatives. Retrieved November 4, 2020.

  * [System And Kernel Extensions In Macos](https://support.apple.com/guide/deployment/system-and-kernel-extensions-in-macos-depa5fb8376f/web), Apple. (n.d.). System and kernel extensions in macOS. Retrieved March 31, 2022.

  * [Github Reptile](https://github.com/f0rb1dd3n/Reptile), Augusto, I. (2018, March 8). Reptile - LMK Linux rootkit. Retrieved April 9, 2018.

  * [Volatility Phalanx2](https://volatility-labs.blogspot.com/2012/10/phalanx-2-revealed-using-volatility-to.html), Case, A. (2012, October 10). Phalanx 2 Revealed: Using Volatility to Analyze an Advanced Linux Rootkit. Retrieved April 9, 2018.

  * [Idefense Rootkit Overview](http://www.megasecurity.org/papers/Rootkits.pdf), Chuvakin, A. (2003, February). An Overview of Rootkits. Retrieved April 6, 2018.

  * [Linux Loadable Kernel Module Insert And Remove Lkms](http://tldp.org/HOWTO/Module-HOWTO/x197.html), Henderson, B. (2006, September 24). How To Insert And Remove LKMs. Retrieved April 9, 2018.

  * [Crowdstrike Linux Rootkit](https://www.crowdstrike.com/blog/http-iframe-injecting-linux-rootkit/), Kurtz, G. (2012, November 19). HTTP iframe Injecting Linux Rootkit. Retrieved December 21, 2017.

  * [Github Diamorphine](https://github.com/m0nad/Diamorphine), Mello, V. (2018, March 8). Diamorphine - LMK rootkit for Linux Kernels 2.6.x/3.x/4.x (x86 and x86_64). Retrieved April 9, 2018.

  * [Securelist Ventir](https://securelist.com/the-ventir-trojan-assemble-your-macos-spy/67267/), Mikhail, K. (2014, October 16). The Ventir Trojan: assemble your MacOS spy. Retrieved April 6, 2018.

  * [User Approved Kernel Extension Pike’S](https://pikeralpha.wordpress.com/2017/08/29/user-approved-kernel-extension-loading/), Pikeralpha. (2017, August 29). User Approved Kernel Extension Loading…. Retrieved September 23, 2021.

  * [Linux Kernel Module Programming Guide](http://www.tldp.org/LDP/lkmpg/2.4/html/x437.html), Pomerantz, O., Salzman, P. (2003, April 4). Modules vs Programs. Retrieved April 6, 2018.

  * [Linux Kernel Programming](https://www.tldp.org/LDP/lkmpg/2.4/lkmpg.pdf), Pomerantz, O., Salzman, P.. (2003, April 4). The Linux Kernel Module Programming Guide. Retrieved April 6, 2018.

  * [Trend Micro Skidmap](https://blog.trendmicro.com/trendlabs-security-intelligence/skidmap-linux-malware-uses-rootkit-capabilities-to-hide-cryptocurrency-mining-payload/), Remillano, A., Urbanec, J. (2019, September 19). Skidmap Linux Malware Uses Rootkit Capabilities to Hide Cryptocurrency-Mining Payload. Retrieved June 4, 2020.

  * [Purves Kextpocalypse 2](https://richard-purves.com/2017/11/09/mdm-and-the-kextpocalypse-2/), Richard Purves. (2017, November 9). MDM and the Kextpocalypse . Retrieved September 23, 2021.

  * [Rsac 2015 San Francisco Patrick Wardle](https://www.virusbulletin.com/uploads/pdf/conference/vb2014/VB2014-Wardle.pdf), Wardle, P. (2015, April). Malware Persistence on OS X Yosemite. Retrieved April 6, 2018.

  * [Synack Secure Kernel Extension Broken](https://www.synack.com/2017/09/08/high-sierras-secure-kernel-extension-loading-is-broken/), Wardle, P. (2017, September 8). High Sierra’s ‘Secure Kernel Extension Loading’ is Broken. Retrieved April 6, 2018.

  * [Wikipedia Loadable Kernel Module](https://en.wikipedia.org/wiki/Loadable_kernel_module#Linux), Wikipedia. (2018, March 17). Loadable kernel module. Retrieved April 9, 2018.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Information Here

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Information Here

## Analytical References

  * [Other references: All custom links should go here](example.lan)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

